In [1]:
import sys, os
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import integrate_vbll
from sklearn.metrics import zero_one_loss, hamming_loss, f1_score, accuracy_score
import torch

from utils import *

vbll found


In [2]:
def setup_data(classes, input_features):
    train_data, train_labels = load_scene(partition='Train')
    train_labels = make_binary(train_labels, classes)

    X_train = train_data[input_features].values
    y_train = train_labels[classes].values

    X_train = X_train[y_train.sum(axis=1) != 0]
    y_train = y_train[y_train.sum(axis=1) != 0]

    test_data, test_labels = load_scene(partition='Test')
    test_labels = make_binary(test_labels, classes)

    X_test = test_data[input_features].values
    y_test = test_labels[classes].values

    X_test = X_test[y_test.sum(axis=1) != 0]
    y_test = y_test[y_test.sum(axis=1) != 0]

    return X_train, y_train, X_test, y_test

In [3]:
classes = ['Beach', 'Sunset', 'FallFoliage', 'Field', 'Mountain', 'Urban']
num_input_features = 294
input_features = ['Att' + str(i) for i in range(1, num_input_features+1)]

X_train, y_train, X_test, y_test = setup_data(classes, input_features)
num_classes = len(classes)
len_data = len(X_train)
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [4]:
model_path = "/home/pyla/bayesian/RBCC/scene_vbll_results_diagonal_1.0_0.001_1000_cuda_1.pt"

reg_weight = 1. / len_data
param = 'diagonal' # diagonal, lowrank, dense
softmax_bound = 'jensen'
return_ood = False
prior_scale = 1.0
noise_label = True

learning_rate = None
num_epochs = None
verbose = True

clf = integrate_vbll.VBLLClassifierClf(num_input_features, batch_size, classes, reg_weight, param, softmax_bound, return_ood, prior_scale, noise_label, device, learning_rate, num_epochs, verbose)
clf.load(model_path)

In [5]:
for h in clf.model_heads:
    print(sum(p.numel() for p in h.parameters()))

1180
1180
1180
1180
1180
1180


In [6]:
train_preds, train_y_pred = clf.predict_with_proba(X_train)
1.0 - zero_one_loss(y_train, train_y_pred)

0.23369116432700254

In [7]:
train_y_pred[:, 0], y_train[:, 0]

(array([0, 0, 0, ..., 0, 0, 0]), array([1., 1., 1., ..., 0., 0., 0.]))

In [8]:
train_y_pred[:, -1].sum(), y_train[:, -1].sum()

(42, 224.0)

In [9]:
for i in range(num_classes):
    print(accuracy_score(y_train[:, i], train_y_pred[:, i]))

0.8538398018166804
0.921552436003303
0.8720066061106524
0.9066886870355079
0.7778695293146161
0.833195706028076


In [10]:
preds, y_pred = clf.predict_with_proba(X_test)

In [11]:
preds[:20]

array([[[0.8322057 , 0.16779432],
        [0.94131845, 0.05868156],
        [0.8999186 , 0.10008141],
        [0.9088417 , 0.09115829],
        [0.7773903 , 0.22260973],
        [0.8654624 , 0.13453764]],

       [[0.6622306 , 0.33776936],
        [0.93485206, 0.06514792],
        [0.8818067 , 0.11819331],
        [0.8967409 , 0.10325911],
        [0.9178033 , 0.08219666],
        [0.8564113 , 0.1435887 ]],

       [[0.77239513, 0.22760479],
        [0.96722364, 0.03277642],
        [0.9278769 , 0.07212314],
        [0.80574703, 0.194253  ],
        [0.66288644, 0.3371136 ],
        [0.8833302 , 0.11666975]],

       [[0.8223967 , 0.17760333],
        [0.95173633, 0.04826362],
        [0.9073658 , 0.0926342 ],
        [0.7608628 , 0.23913723],
        [0.8216607 , 0.17833932],
        [0.9123783 , 0.0876217 ]],

       [[0.710456  , 0.28954402],
        [0.9623254 , 0.03767462],
        [0.93216974, 0.06783025],
        [0.9159406 , 0.08405946],
        [0.71241516, 0.28758487],
      

In [12]:
y_pred[:20]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [13]:
y_test[:20]

array([[1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [17]:
train_y_pred[:, -1].sum(), y_train[:, -1].sum()

(1137, 224.0)

In [18]:
learnable_parameters = [param for model in clf.model_heads for param in model.parameters()]
print("Number of (learnable) parameters in the model: ", sum(p.numel() for p in learnable_parameters if p.requires_grad))

Number of (learnable) parameters in the model:  7068


In [25]:
model_path = "/home/pyla/bayesian/RBCC/scene_vbll_extended_results_294x0_diagonal_1.0_0.001_1000_cuda_0.pt"

hidden_size = 294
n_hidden = 0

reg_weight = 1. / len_data
param = 'diagonal' # diagonal, lowrank, dense
softmax_bound = 'jensen'
return_ood = False
prior_scale = 1.0
noise_label = True

learning_rate = None
num_epochs = None
verbose = True

extended_clf = integrate_vbll.ExtendedVBLLClassifierClf(num_input_features, hidden_size, n_hidden, batch_size, classes, reg_weight, param, softmax_bound, return_ood, prior_scale, noise_label, device, learning_rate, num_epochs, verbose)
extended_clf.load(model_path)

In [26]:
for i in range(num_classes):
    print(accuracy_score(y_train[:, i], train_y_pred[:, i]))
train_preds, train_y_pred = extended_clf.predict_with_proba(X_train)
1.0 - zero_one_loss(y_train, train_y_pred)

0.7481420313790256
0.2815854665565648
0.4541701073492981
0.7927332782824112
0.4153592072667217
0.20644095788604458


0.03633360858794388

In [27]:
y_train[:10]

array([[1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [30]:
preds[:10]

array([[[0.5547686 , 0.44523138],
        [0.4138832 , 0.5861168 ],
        [0.45767698, 0.542323  ],
        [0.55167854, 0.44832143],
        [0.5243117 , 0.47568828],
        [0.4601263 , 0.5398737 ]],

       [[0.4207653 , 0.57923466],
        [0.47640228, 0.5235977 ],
        [0.5005409 , 0.4994591 ],
        [0.5065948 , 0.49340525],
        [0.47373027, 0.52626973],
        [0.45753407, 0.542466  ]],

       [[0.59425104, 0.405749  ],
        [0.4571649 , 0.5428351 ],
        [0.47724557, 0.5227545 ],
        [0.57728773, 0.42271227],
        [0.54566634, 0.45433363],
        [0.4816397 , 0.51836026]],

       [[0.6760004 , 0.32399955],
        [0.48609647, 0.5139035 ],
        [0.49518928, 0.5048107 ],
        [0.56318176, 0.43681827],
        [0.5193413 , 0.48065874],
        [0.4709327 , 0.52906734]],

       [[0.6032166 , 0.3967834 ],
        [0.48791614, 0.5120839 ],
        [0.4785679 , 0.52143216],
        [0.53808486, 0.4619151 ],
        [0.51285535, 0.48714462],
      

In [28]:
train_y_pred[:10]

array([[0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 1, 0]])

In [16]:
preds, y_pred = extended_clf.predict_with_proba(X_test)
preds, y_pred

(array([[[0.5547686 , 0.44523138],
         [0.4138832 , 0.5861168 ],
         [0.45767698, 0.542323  ],
         [0.55167854, 0.44832143],
         [0.5243117 , 0.47568828],
         [0.4601263 , 0.5398737 ]],
 
        [[0.4207653 , 0.57923466],
         [0.47640228, 0.5235977 ],
         [0.5005409 , 0.4994591 ],
         [0.5065948 , 0.49340525],
         [0.47373027, 0.52626973],
         [0.45753407, 0.542466  ]],
 
        [[0.59425104, 0.405749  ],
         [0.4571649 , 0.5428351 ],
         [0.47724557, 0.5227545 ],
         [0.57728773, 0.42271227],
         [0.54566634, 0.45433363],
         [0.4816397 , 0.51836026]],
 
        ...,
 
        [[0.60943997, 0.39056006],
         [0.4442079 , 0.55579215],
         [0.53572893, 0.46427107],
         [0.54193825, 0.45806172],
         [0.52433264, 0.4756674 ],
         [0.44143742, 0.5585626 ]],
 
        [[0.601758  , 0.39824206],
         [0.45604628, 0.5439537 ],
         [0.54080236, 0.4591977 ],
         [0.58369696, 0.4163

In [19]:
learnable_parameters = [param for model in extended_clf.model_heads for param in model.parameters()]
if extended_clf.model_base is not None:
    learnable_parameters += [param for param in extended_clf.model_base.parameters()]
print("Number of (learnable) parameters in the model: ", sum(p.numel() for p in learnable_parameters if p.requires_grad))

Number of (learnable) parameters in the model:  93798


In [22]:
import classifiers

input_size = len(input_features)
hidden_size = 100
parent_dict = {
    0: {'class': 'Beach', 'parents': []},
    1: {'class': 'Sunset', 'parents': []},
    2: {'class': 'FallFoliage', 'parents': []},
    3: {'class': 'Field', 'parents': []},
    4: {'class': 'Mountain', 'parents': []},
    5: {'class': 'Urban', 'parents': []}
}

clf_bayrnn = classifiers.BayClassifierChainClf(input_size, hidden_size, num_classes, batch_size, parent_dict, classes, device, learning_rate, num_epochs, verbose=True)

sum(p.numel() for p in clf_bayrnn.model.parameters())

121206